In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

train_data= pd.read_csv('/Users/asthagupta/Desktop/faltoo folder/mnist_train.csv')
len(train_data)

In [ ]:
def batch_generator(data, batch_size, num_epochs):
    current_epoch = 0
    
    while current_epoch < num_epochs:
        # Shuffle at the start of each epoch
        shuffled_data = data.sample(frac=1).reset_index(drop=True)
        num_samples = len(shuffled_data)
        no_of_batches = num_samples // batch_size
        # Generate batches for current epoch
        for start_idx in range(1, no_of_batches + 1):
            end_idx = min(start_idx + batch_size, num_samples)
            batch = shuffled_data.iloc[start_idx:end_idx]
            
            # Prepare features and labels
            X_train = batch.drop(columns=['label']).values.T  # Transpose features
            y_train = batch['label'].values
            
            yield X_train, y_train, current_epoch
            
        current_epoch += 1




generator = batch_generator(train_data,10000,2)
for X_train, y_train, current_epoch in generator:
    print(f"Epoch: {current_epoch}, Batch shape: {X_train},{y_train}")
    # Use X_batch and y_batch in your training loop

X~ 784 * m
w1 ~ 16*784
b1 ~ 16*1
w2 ~ 10*16
b2 ~ 10*1

In [ ]:
# initial parameters
def parameter_initialization():
    
    W1 = np.random.randn(16,784)-0.5
    W2 = np.random.randn(10, 16)-0.5
    b1 = np.random.randn(16,1)-0.5
    b2 = np.random.randn(10,1)-0.5
    return W1, W2, b1, b2

In [ ]:
# relu function
def relu(x):
    return np.maximum(0, x)
# derivative of relu
def relu_derivative(x):
    return np.where(x > 0, 1, 0)

we are subtracting np.max(x) to maintain stability 
keepdims=TRUE so that the original dimension is maintained

In [18]:
# softmax function
def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum(axis=0, keepdims=True)
softmax(np.array([[1, 2, 3], [4, 5, 6]]))

array([[0.04742587, 0.04742587, 0.04742587],
       [0.95257413, 0.95257413, 0.95257413]])

In [17]:

# one hot encoding
def one_hot_encode(y, m):
    y_encoded = np.zeros((10,m))
    y_encoded[y, np.arange(m)] = 1
    return y_encoded

one_hot_encode(np.array([1, 2, 3]), 3)

array([[0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [ ]:
# forward propagation
def forward(X, W1, b1, W2, b2):
    Z1=np.dot(W1*X)+b1
    A1=relu(Z1)
    Z2=np.dot(W2*A1)+b2
    A2=softmax(Z2)
    return A1, A2,Z1,Z2

loss function =

In [ ]:
# backward propagation
def backward(X, Y, A1, A2, W2, Z1, Z2,m):
    
    dZ2 = A2 - Y
    dW2 = 1/m*np.dot(dZ2,A1.T) 
    db2 = 1/m*np.sum(dZ2, axis=0) 

    dZ1 = 1/m*np.dot(dZ2 ) * relu_derivative(Z1)
    dW1 = 1/m*np.dot(dZ1, X.T) 
    db1 = 1/m*np.sum(dZ1, axis=0) 
    return dW1, db1, dW2, db2

In [ ]:
def gradient_descent(X, Y, W1, b1, W2, b2, learning_rate=0.01):
    m = X.shape[0]
    A1, A2,Z1,Z2 = forward(X, W1, b1, W2, b2)
    dW1, db1, dW2, db2 = backward(X, Y, A1, A2, W2, Z1, Z2,m)
    
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    
    return W1,b1,W2,b2